In [2]:
import yt.wrapper as yt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu, ttest_ind
from collections import Counter
import os

from tqdm.auto import tqdm

import json


%matplotlib inline

## Основные числа

Всего 197 сессий (95 из теста и 102 из контроля)

94 согласованных в разметке сессионного качества

111 согласованных в разметке контекста

In [3]:
yt.config["proxy"]["url"] = 'hahn.yt.yandex.net'
client = yt.YtClient(proxy='hahn', token=os.environ.get('YT_TOKEN'))

In [56]:
toloka_results = pd.read_csv("toloka_results_2.tsv", sep="\t")
ilnur_table = read_result = pd.DataFrame(
    client.read_table("//home/voice/ilnur/tasks/EXPERIMENTS-73207/extract_result_2_portions", raw=False)
)

In [57]:
toloka_results["session_id"] = toloka_results["INPUT:input"].apply(
    lambda x: json.loads(x)["session_id"]
)

In [58]:
toloka_results

,INPUT:input,OUTPUT:result,OUTPUT:comment,OUTPUT:context,GOLDEN:result,GOLDEN:comment,GOLDEN:context,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,session_id
0,"{""action0"":{""action"":""Включается радио \""Европ...",bad,"Дважды спросила про то, как день пользователя,...",bad,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1165159...,000011c767--626a6ad2c0295242c1f455a3,000011c767--626fb5904a1e946093d1c4d1,62d3f41254aee9b8522e2453511ba80f,APPROVED,2022-05-02T10:42:24.599,uu/2fbdf2189c67b8c7fbbdc308d3302ddd_1624866502...
1,"{""action0"":{""answer"":""Могу ошибаться, но на им...",bad,NaN,zero,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1165159...,000011c767--626a6ad2c0295242c1f454c8,000011c767--626fb751dbe4e9761d7f09c7,62d3f41254aee9b8522e2453511ba80f,APPROVED,2022-05-02T10:49:53.528,uu/3eba99ac8e43f2f3be850327640e1804_1625652612...
2,"{""action0"":{""action"":""Первый трек, который вкл...",stupid,с точки зрения именно болталочного диалога тут...,bad,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1165159...,000011c767--626a6ad2c0295242c1f45587,000011c767--626fb79c6417d63fa6486269,62d3f41254aee9b8522e2453511ba80f,APPROVED,2022-05-02T10:51:08.806,uu/c02fdb63db4fd8a20cc4a7001e8a2182_1624820110...
3,"{""action0"":{""action"":""Музыка и/или видео ставя...",bad,"Диалог получился тупой, пользователь не понял,...",bad,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1165159...,000011c767--626a6ad2c0295242c1f454b2,000011c767--626fb9083e1f6f02bc8abc89,62d3f41254aee9b8522e2453511ba80f,APPROVED,2022-05-02T10:57:12.896,uu/d31d7f0dac97a9342f3aea2fbf818788_1626228286...
4,"{""action0"":{""action"":""Яркость устройства Лампо...",part,Плохо обрезали сессию в диалоге. Тут бы больше...,zero,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1165159...,000011c767--626a6ad2c0295242c1f4556d,000011c767--626fbae06417d63fa6486296,62d3f41254aee9b8522e2453511ba80f,APPROVED,2022-05-02T11:05:04.305,uu/9988fe82d37e28cfeb2d2d61aabb968d_1624803035...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,"{""action0"":{""answer"":""Так я еще только приступ...",part,NaN,good,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1165159...,000011c767--626a6ad2c0295242c1f45586,000011c767--628158df80af5113d94719dd,9f1f4f052ebd5b1cd9366cabcd3fa392,APPROVED,2022-05-15T19:47:43.282,uu/c774fde1c6f809741c35bd7cfeb09081_1624640345...
393,"{""action0"":{""action"":""Музыка и/или видео ставя...",part,NaN,bad,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1165159...,000011c767--626a6ad2c0295242c1f455c0,000011c767--6281595780af5113d94719ef,9f1f4f052ebd5b1cd9366cabcd3fa392,APPROVED,2022-05-15T19:49:43.339,uu/6207068a98c8154996d9f6f8210c0e9b_1626436298...
394,"{""action0"":{""answer"":""Спасибо. Это пригодится....",good,NaN,zero,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1165159...,000011c767--626a6ad2c0295242c1f454fe,000011c767--6281599d25f30e1af03ea456,9f1f4f052ebd5b1cd9366cabcd3fa392,APPROVED,2022-05-15T19:50:53.060,uu/e51c4834e9e36135a738dff0d61c2b46_1624963914...
395,"{""action0"":{""answer"":""Сейчас в Красноярске +20...",good,NaN,good,NaN,NaN,NaN,NaN,NaN,https://sandbox.toloka.yandex.com/task/1165159...,000011c767--626a6ad2c0295242c1f45530,000011c767--628159ba3953fa144fc0c92a,9f1f4f052ebd5b1cd9366cabcd3fa392,APPROVED,2022-05-15T19:51:22.790,uu/b4b50203eb71a49c7b26f292e0a54d98_1626410852...


In [59]:
toloka_results["is_bad"] = 1*(toloka_results["OUTPUT:result"] == "bad")
toloka_results["is_good"] = 1*(toloka_results["OUTPUT:result"] == "good")
toloka_results["is_part"] = 1*(toloka_results["OUTPUT:result"] == "part")
toloka_results["is_stupid"] = 1*(toloka_results["OUTPUT:result"] == "stupid")


toloka_results["is_bad_context"] = 1*(toloka_results["OUTPUT:context"] == "bad")
toloka_results["is_zero_context"] = 1*(toloka_results["OUTPUT:context"] == "zero")
toloka_results["is_good_context"] = 1*(toloka_results["OUTPUT:context"] == "good")

In [60]:
aggregated_results = toloka_results.groupby(by=["session_id"]).agg(
                                            part_cnt = pd.NamedAgg(column = 'is_part', aggfunc = 'sum'),
                                            stupid_cnt = pd.NamedAgg(column = 'is_stupid', aggfunc = 'sum'),
                                            good_cnt = pd.NamedAgg(column = 'is_good', aggfunc = 'sum'),
                                            bad_cnt = pd.NamedAgg(column = 'is_bad', aggfunc = 'sum'),
                                            zero_context_cnt = pd.NamedAgg(column = 'is_zero_context', aggfunc = 'sum'),
                                            good_context_cnt = pd.NamedAgg(column = 'is_good_context', aggfunc = 'sum'),
                                            bad_context_cnt = pd.NamedAgg(column = 'is_bad_context', aggfunc = 'sum'),
        
                                            ).reset_index()

In [61]:
aggregated_results["session_id_old"] = aggregated_results["session_id"].apply(
    lambda x: "_".join(x.split("_")[:4])
)

In [62]:
aggregated_results.merge(ilnur_table, on="session_id_old")

,session_id,part_cnt,stupid_cnt,good_cnt,bad_cnt,zero_context_cnt,good_context_cnt,bad_context_cnt,session_id_old,frequency,portion,result,session_len,target_scenario,target_scenario_rounded,test_name,uuid
0,uu/0233b4b22da8dbd029ddd04a95a1e94d_1624705868...,0,1,0,1,2,0,0,uu/0233b4b22da8dbd029ddd04a95a1e94d_1624705868...,321,portion1,part,16,12,12,test,uu/0233b4b22da8dbd029ddd04a95a1e94d
1,uu/029e9aba04f5fbe82ae9659c293a1b2e_1626527294...,0,0,0,2,2,0,0,uu/029e9aba04f5fbe82ae9659c293a1b2e_1626527294...,503,portion1,part,8,6,6,test,uu/029e9aba04f5fbe82ae9659c293a1b2e
2,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,1,0,1,0,0,1,1,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,4980,portion1,part,36,32,16,ctrl,uu/0371ea102b5cd47ff8396c5f62a83a7d
3,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,1,0,1,0,0,1,1,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,4980,portion1,part,36,32,16,ctrl,uu/0371ea102b5cd47ff8396c5f62a83a7d
4,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,1,0,1,0,0,1,1,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,4980,portion1,stupid,36,32,16,ctrl,uu/0371ea102b5cd47ff8396c5f62a83a7d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,uu/f68f4f582a29e9e877f9bec30c8a4dbf_1624720185...,1,0,0,1,0,0,2,uu/f68f4f582a29e9e877f9bec30c8a4dbf_1624720185...,386,portion1,good,13,9,9,test,uu/f68f4f582a29e9e877f9bec30c8a4dbf
401,uu/faa28b2cf6bc40d44b89373be398e466_1626937202...,0,0,2,0,1,1,0,uu/faa28b2cf6bc40d44b89373be398e466_1626937202...,4805,portion1,good,25,21,16,test,uu/faa28b2cf6bc40d44b89373be398e466
402,uu/faa28b2cf6bc40d44b89373be398e466_1626937202...,0,0,2,0,1,1,0,uu/faa28b2cf6bc40d44b89373be398e466_1626937202...,4805,portion1,bad,25,21,16,test,uu/faa28b2cf6bc40d44b89373be398e466
403,uu/faa28b2cf6bc40d44b89373be398e466_1626937202...,0,0,0,2,1,0,1,uu/faa28b2cf6bc40d44b89373be398e466_1626937202...,4805,portion1,good,25,21,16,test,uu/faa28b2cf6bc40d44b89373be398e466


In [63]:
test_id = ilnur_table.groupby(by="session_id_old").agg(
    test_name = pd.NamedAgg(column = 'test_name', aggfunc = 'last'),
    session_len = pd.NamedAgg(column = 'session_len', aggfunc = 'last'),
).reset_index()

In [64]:
aggregated_results = aggregated_results.merge(test_id, on="session_id_old")

In [65]:
only_with_2_assigners = aggregated_results[
    aggregated_results["part_cnt"] + aggregated_results["stupid_cnt"] + aggregated_results["good_cnt"] + aggregated_results["bad_cnt"] > 1
].reset_index(drop=True)

In [66]:
Counter(only_with_2_assigners["test_name"])

Counter({'test': 95, 'ctrl': 102})

In [67]:
only_with_2_assigners

,session_id,part_cnt,stupid_cnt,good_cnt,bad_cnt,zero_context_cnt,good_context_cnt,bad_context_cnt,session_id_old,test_name,session_len
0,uu/0233b4b22da8dbd029ddd04a95a1e94d_1624705868...,0,1,0,1,2,0,0,uu/0233b4b22da8dbd029ddd04a95a1e94d_1624705868...,test,16
1,uu/029e9aba04f5fbe82ae9659c293a1b2e_1626527294...,0,0,0,2,2,0,0,uu/029e9aba04f5fbe82ae9659c293a1b2e_1626527294...,test,8
2,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,1,0,1,0,0,1,1,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,ctrl,36
3,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,2,0,0,0,0,1,1,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,ctrl,36
4,uu/03b238aa7b0f9a7078e448aadacffe33_1626791571...,1,0,0,1,1,0,1,uu/03b238aa7b0f9a7078e448aadacffe33_1626791571...,test,20
...,...,...,...,...,...,...,...,...,...,...,...
192,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,1,0,1,0,2,0,0,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,ctrl,16
193,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,0,0,2,0,0,2,0,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,ctrl,16
194,uu/f68f4f582a29e9e877f9bec30c8a4dbf_1624720185...,1,0,0,1,0,0,2,uu/f68f4f582a29e9e877f9bec30c8a4dbf_1624720185...,test,13
195,uu/faa28b2cf6bc40d44b89373be398e466_1626937202...,0,0,2,0,1,1,0,uu/faa28b2cf6bc40d44b89373be398e466_1626937202...,test,25


In [68]:
only_with_2_assigners[
    (only_with_2_assigners["part_cnt"] > 1) | 
    (only_with_2_assigners["stupid_cnt"] > 1)|
    (only_with_2_assigners["good_cnt"] > 1) |
    (only_with_2_assigners["bad_cnt"] > 1)
]

,session_id,part_cnt,stupid_cnt,good_cnt,bad_cnt,zero_context_cnt,good_context_cnt,bad_context_cnt,session_id_old,test_name,session_len
1,uu/029e9aba04f5fbe82ae9659c293a1b2e_1626527294...,0,0,0,2,2,0,0,uu/029e9aba04f5fbe82ae9659c293a1b2e_1626527294...,test,8
3,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,2,0,0,0,0,1,1,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,ctrl,36
6,uu/093bd83d9a4cd21adb0de82ac97861be_1624813234...,2,0,0,0,1,1,0,uu/093bd83d9a4cd21adb0de82ac97861be_1624813234...,ctrl,47
7,uu/0b74b6984b24adcde0501c9030a2b712_1626175994...,0,0,0,2,1,1,0,uu/0b74b6984b24adcde0501c9030a2b712_1626175994...,ctrl,18
9,uu/0e85b2233afab3a0782c7277accb628e_1624954544...,0,0,0,2,1,0,1,uu/0e85b2233afab3a0782c7277accb628e_1624954544...,test,10
...,...,...,...,...,...,...,...,...,...,...,...
190,uu/f2559895ad6d25c40a04b2e60e8fcfc0_1625588107...,0,0,2,0,0,2,0,uu/f2559895ad6d25c40a04b2e60e8fcfc0_1625588107...,ctrl,9
191,uu/f3b0e87247bd9322efbcfce67534204b_1624857580...,2,0,0,0,1,1,0,uu/f3b0e87247bd9322efbcfce67534204b_1624857580...,ctrl,12
193,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,0,0,2,0,0,2,0,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,ctrl,16
195,uu/faa28b2cf6bc40d44b89373be398e466_1626937202...,0,0,2,0,1,1,0,uu/faa28b2cf6bc40d44b89373be398e466_1626937202...,test,25


In [69]:
only_with_2_assigners[
    (only_with_2_assigners["zero_context_cnt"] > 1) | 
    (only_with_2_assigners["good_context_cnt"] > 1)|
    (only_with_2_assigners["bad_context_cnt"] > 1)
]

,session_id,part_cnt,stupid_cnt,good_cnt,bad_cnt,zero_context_cnt,good_context_cnt,bad_context_cnt,session_id_old,test_name,session_len
0,uu/0233b4b22da8dbd029ddd04a95a1e94d_1624705868...,0,1,0,1,2,0,0,uu/0233b4b22da8dbd029ddd04a95a1e94d_1624705868...,test,16
1,uu/029e9aba04f5fbe82ae9659c293a1b2e_1626527294...,0,0,0,2,2,0,0,uu/029e9aba04f5fbe82ae9659c293a1b2e_1626527294...,test,8
8,uu/0d002096b5b45410df7dda221d6e03f7_1626187607...,0,0,1,1,2,0,0,uu/0d002096b5b45410df7dda221d6e03f7_1626187607...,test,10
10,uu/123ebc298ddbb2f4cc81ad59e4684b18_1626344403...,1,0,1,0,2,0,0,uu/123ebc298ddbb2f4cc81ad59e4684b18_1626344403...,ctrl,58
11,uu/123ebc298ddbb2f4cc81ad59e4684b18_1626344403...,0,0,1,1,0,2,0,uu/123ebc298ddbb2f4cc81ad59e4684b18_1626344403...,ctrl,58
...,...,...,...,...,...,...,...,...,...,...,...
188,uu/edc8fe0eaa95bfe8f670df20ecd626b2_1625830500...,0,0,2,0,2,0,0,uu/edc8fe0eaa95bfe8f670df20ecd626b2_1625830500...,test,6
190,uu/f2559895ad6d25c40a04b2e60e8fcfc0_1625588107...,0,0,2,0,0,2,0,uu/f2559895ad6d25c40a04b2e60e8fcfc0_1625588107...,ctrl,9
192,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,1,0,1,0,2,0,0,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,ctrl,16
193,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,0,0,2,0,0,2,0,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,ctrl,16


In [70]:
pairs_in_sessions = []
pairs_in_context = []

for i, row in only_with_2_assigners.iterrows():
    marks = ["good"] * row["good_cnt"] + ["part"] * row["part_cnt"] + ["bad"] * row["bad_cnt"] + ["stupid"] * row["stupid_cnt"]
    
    pairs_in_sessions.append(" ".join(marks))
    
    context_marks = ["good"] * row["good_context_cnt"]+ ["zero"] * row["zero_context_cnt"]+ ["bad"] * row["bad_context_cnt"]
    
    pairs_in_context.append(" ".join(context_marks))
    

In [71]:
only_with_2_assigners["pairs_in_sessions"] = pairs_in_sessions
only_with_2_assigners["pairs_in_context"] = pairs_in_context

In [72]:
only_with_2_assigners

,session_id,part_cnt,stupid_cnt,good_cnt,bad_cnt,zero_context_cnt,good_context_cnt,bad_context_cnt,session_id_old,test_name,session_len,pairs_in_sessions,pairs_in_context
0,uu/0233b4b22da8dbd029ddd04a95a1e94d_1624705868...,0,1,0,1,2,0,0,uu/0233b4b22da8dbd029ddd04a95a1e94d_1624705868...,test,16,bad stupid,zero zero
1,uu/029e9aba04f5fbe82ae9659c293a1b2e_1626527294...,0,0,0,2,2,0,0,uu/029e9aba04f5fbe82ae9659c293a1b2e_1626527294...,test,8,bad bad,zero zero
2,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,1,0,1,0,0,1,1,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,ctrl,36,good part,good bad
3,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,2,0,0,0,0,1,1,uu/0371ea102b5cd47ff8396c5f62a83a7d_1626879903...,ctrl,36,part part,good bad
4,uu/03b238aa7b0f9a7078e448aadacffe33_1626791571...,1,0,0,1,1,0,1,uu/03b238aa7b0f9a7078e448aadacffe33_1626791571...,test,20,part bad,zero bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,1,0,1,0,2,0,0,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,ctrl,16,good part,zero zero
193,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,0,0,2,0,0,2,0,uu/f68d933770a7d9e7aec2360b3b65c204_1625065571...,ctrl,16,good good,good good
194,uu/f68f4f582a29e9e877f9bec30c8a4dbf_1624720185...,1,0,0,1,0,0,2,uu/f68f4f582a29e9e877f9bec30c8a4dbf_1624720185...,test,13,part bad,bad bad
195,uu/faa28b2cf6bc40d44b89373be398e466_1626937202...,0,0,2,0,1,1,0,uu/faa28b2cf6bc40d44b89373be398e466_1626937202...,test,25,good good,good zero


In [73]:
session_cnt = dict(Counter(pairs_in_sessions))
df_session = pd.DataFrame({
    "pair_of_answers" : session_cnt.keys(),
    "cnt" : session_cnt.values(),
})

context_cnt = dict(Counter(pairs_in_context))
df_context = pd.DataFrame({
    "pair_of_answers" : context_cnt.keys(),
    "cnt" : context_cnt.values(),
})

In [74]:
import plotly.express as px

# fig = px.pie(df_session, values='cnt', names='pair_of_answers', title='Пары ответов в разметке сессионного качества')
# fig.update_traces(textposition='inside', textinfo='percent+label')
fig = px.bar(df_session, y='cnt', x='pair_of_answers', title='Пары ответов в разметке сессионного качества')
fig.show()

In [114]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=2)


answers = ["good", "part", "bad", "stupid"]

for i, ans in enumerate(answers):
    
    cnt = df_session[df_session["pair_of_answers"].apply(lambda x: ans in x)]
    
    
    fig.add_trace(
         go.Bar(
                 y=cnt["cnt"],
                 x=cnt["pair_of_answers"], 
                 ),
        row=1 + i//2, col=1 + i%2
    )

fig.update_layout(height=600, width=800, title="Пары ответов сгрупированные по первому ответу")
fig.show()

In [75]:
# fig = px.pie(df_context, values='cnt', names='pair_of_answers', title='Пары ответов в разметке контекста')
# fig.update_traces(textposition='inside', textinfo='percent+label')
fig = px.bar(df_context, y='cnt', x='pair_of_answers', title='Пары ответов в разметке контекста')
fig.show()

In [115]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=3)


answers = ["good", "zero", "bad"]

for i, ans in enumerate(answers):
    
    cnt = df_context[df_context["pair_of_answers"].apply(lambda x: ans in x)]
    
    
    fig.add_trace(
         go.Bar(
                 y=cnt["cnt"],
                 x=cnt["pair_of_answers"], 
                 ),
        row=1, col=1 + i%3
    )

fig.update_layout(height=600, width=800, title="Пары ответов в разметке контекста ответов сгрупированные по первому ответу")
fig.show()

In [76]:
def make_test(data, metrica_column, filter_stupid=False):
    flag = True
    
    if filter_stupid: 
        flag = flag & (data["stupid_cnt"] == 0)
    
    test_raw = data[flag & (data["test_name"] == "test")][metrica_column]
    ctrl_raw = data[flag & (data["test_name"] == "ctrl")][metrica_column]
    
    
    pvalue=mannwhitneyu(ctrl_raw, test_raw).pvalue
    effect = (test_raw.mean() - ctrl_raw.mean())/ctrl_raw.mean()
    message = f'metrica={metrica_column}, pvalue={pvalue:.3f}, effect={effect:.3f}, ctrl metric = {ctrl_raw.mean():.3f}, test_metric = {test_raw.mean():.3f}, cnt = {len(ctrl_raw) + len(test_raw)}'
    #, pvalue, effect, ctrl_metric.mean(), test_metric.mean()
    
    print(message)
    

In [77]:
only_with_2_assigners["avg_metrica"] = ((
    only_with_2_assigners["good_cnt"] + 0.5 * only_with_2_assigners["part_cnt"] -  only_with_2_assigners["stupid_cnt"])
    / (only_with_2_assigners["good_cnt"] + only_with_2_assigners["part_cnt"] + only_with_2_assigners["stupid_cnt"] + only_with_2_assigners["bad_cnt"])
)

only_with_2_assigners["part_of_bad"] = ((
    only_with_2_assigners["bad_cnt"])
    / (only_with_2_assigners["good_cnt"] + only_with_2_assigners["part_cnt"] + only_with_2_assigners["stupid_cnt"] + only_with_2_assigners["bad_cnt"])
)

only_with_2_assigners["part_of_not_bad"] = ((
    only_with_2_assigners["good_cnt"] + only_with_2_assigners["part_cnt"])
    / (only_with_2_assigners["good_cnt"] + only_with_2_assigners["part_cnt"] + only_with_2_assigners["stupid_cnt"] + only_with_2_assigners["bad_cnt"])
)

only_with_2_assigners["part_of_good"] = ((
    only_with_2_assigners["good_cnt"])
    / (only_with_2_assigners["good_cnt"] + only_with_2_assigners["part_cnt"] + only_with_2_assigners["stupid_cnt"] + only_with_2_assigners["bad_cnt"])
)


only_with_2_assigners["avg_metrica_context"] = ((
    only_with_2_assigners["good_context_cnt"] -  only_with_2_assigners["bad_context_cnt"])
    / (only_with_2_assigners["zero_context_cnt"]+ only_with_2_assigners["good_context_cnt"] + only_with_2_assigners["bad_context_cnt"])
)

only_with_2_assigners["part_of_bad_context"] = ((
    only_with_2_assigners["bad_context_cnt"])
    / (only_with_2_assigners["zero_context_cnt"]+ only_with_2_assigners["good_context_cnt"] + only_with_2_assigners["bad_context_cnt"])
)

only_with_2_assigners["part_of_not_bad_context"] = ((
    only_with_2_assigners["zero_context_cnt"] + only_with_2_assigners["good_context_cnt"])
    / (only_with_2_assigners["zero_context_cnt"]+ only_with_2_assigners["good_context_cnt"] + only_with_2_assigners["bad_context_cnt"])
)

only_with_2_assigners["part_of_good_context"] = ((
    only_with_2_assigners["good_context_cnt"])
    / (only_with_2_assigners["zero_context_cnt"]+ only_with_2_assigners["good_context_cnt"] + only_with_2_assigners["bad_context_cnt"])
)
   

In [78]:
make_test(only_with_2_assigners, "avg_metrica")
make_test(only_with_2_assigners, "avg_metrica", filter_stupid=True)

metrica=avg_metrica, pvalue=0.969, effect=-0.090, ctrl metric = 0.306, test_metric = 0.279, cnt = 197
metrica=avg_metrica, pvalue=0.801, effect=-0.026, ctrl metric = 0.440, test_metric = 0.429, cnt = 165


In [79]:
make_test(only_with_2_assigners, "part_of_bad")
make_test(only_with_2_assigners, "part_of_bad", filter_stupid=True)

metrica=part_of_bad, pvalue=0.699, effect=0.057, ctrl metric = 0.319, test_metric = 0.337, cnt = 197
metrica=part_of_bad, pvalue=0.704, effect=0.054, ctrl metric = 0.357, test_metric = 0.377, cnt = 165


In [80]:
make_test(only_with_2_assigners, "part_of_not_bad")
make_test(only_with_2_assigners, "part_of_not_bad", filter_stupid=True)

metrica=part_of_not_bad, pvalue=0.665, effect=-0.044, ctrl metric = 0.583, test_metric = 0.558, cnt = 197
metrica=part_of_not_bad, pvalue=0.704, effect=-0.030, ctrl metric = 0.643, test_metric = 0.623, cnt = 165


In [81]:
make_test(only_with_2_assigners, "part_of_good")
make_test(only_with_2_assigners, "part_of_good", filter_stupid=True)

metrica=part_of_good, pvalue=0.775, effect=-0.066, ctrl metric = 0.225, test_metric = 0.211, cnt = 197
metrica=part_of_good, pvalue=0.958, effect=-0.015, ctrl metric = 0.238, test_metric = 0.235, cnt = 165


In [82]:
make_test(only_with_2_assigners, "avg_metrica_context")
make_test(only_with_2_assigners, "avg_metrica_context", filter_stupid=True)

metrica=avg_metrica_context, pvalue=0.360, effect=1.224, ctrl metric = -0.069, test_metric = -0.153, cnt = 197
metrica=avg_metrica_context, pvalue=0.441, effect=1.074, ctrl metric = -0.077, test_metric = -0.160, cnt = 165


In [83]:
make_test(only_with_2_assigners, "part_of_bad_context")
make_test(only_with_2_assigners, "part_of_bad_context", filter_stupid=True)

metrica=part_of_bad_context, pvalue=0.787, effect=-0.030, ctrl metric = 0.304, test_metric = 0.295, cnt = 197
metrica=part_of_bad_context, pvalue=0.734, effect=-0.054, ctrl metric = 0.339, test_metric = 0.321, cnt = 165


In [84]:
make_test(only_with_2_assigners, "part_of_not_bad_context")
make_test(only_with_2_assigners, "part_of_not_bad_context", filter_stupid=True)

metrica=part_of_not_bad_context, pvalue=0.787, effect=0.013, ctrl metric = 0.696, test_metric = 0.705, cnt = 197
metrica=part_of_not_bad_context, pvalue=0.734, effect=0.028, ctrl metric = 0.661, test_metric = 0.679, cnt = 165


In [85]:
make_test(only_with_2_assigners, "part_of_good_context")
make_test(only_with_2_assigners, "part_of_good_context", filter_stupid=True)

metrica=part_of_good_context, pvalue=0.048, effect=-0.396, ctrl metric = 0.235, test_metric = 0.142, cnt = 197
metrica=part_of_good_context, pvalue=0.065, effect=-0.387, ctrl metric = 0.262, test_metric = 0.160, cnt = 165


In [86]:
only_with_2_assigners["session_len"]

0      16
1       8
2      36
3      36
4      20
       ..
192    16
193    16
194    13
195    25
196    25
Name: session_len, Length: 197, dtype: int64

In [87]:
session_len_grouped = only_with_2_assigners.groupby(by=["session_len", "test_name"]).agg(
                                            zero_context_cnt = pd.NamedAgg(column = 'zero_context_cnt', aggfunc = 'sum'),
                                            good_context_cnt = pd.NamedAgg(column = 'good_context_cnt', aggfunc = 'sum'),
                                            bad_context_cnt = pd.NamedAgg(column = 'bad_context_cnt', aggfunc = 'sum'),
).reset_index()

In [88]:
session_len_grouped["cnt"] = session_len_grouped["zero_context_cnt"] + session_len_grouped["good_context_cnt"] + session_len_grouped["bad_context_cnt"]

session_len_grouped["part_of_good"] = session_len_grouped["good_context_cnt"] / session_len_grouped["cnt"]
session_len_grouped["std_part_of_good"] =session_len_grouped["part_of_good"] * ( 1 / session_len_grouped["good_context_cnt"]
                                                                                + 1 / session_len_grouped["cnt"])

session_len_grouped["part_of_bad"] = session_len_grouped["bad_context_cnt"] / session_len_grouped["cnt"]
session_len_grouped["std_part_of_bad"] =session_len_grouped["part_of_bad"] * ( 1 / session_len_grouped["bad_context_cnt"] + 1 / session_len_grouped["cnt"])

session_len_grouped["part_of_not_bad"] = 1 - session_len_grouped["bad_context_cnt"] / session_len_grouped["cnt"]
session_len_grouped["std_part_of_not_bad"] = session_len_grouped["std_part_of_bad"]
session_len_grouped["log_part_of_not_bad"] = np.log(session_len_grouped["part_of_not_bad"])

session_len_grouped

/Users/eliseevmax/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning:

divide by zero encountered in log



,session_len,test_name,zero_context_cnt,good_context_cnt,bad_context_cnt,cnt,part_of_good,std_part_of_good,part_of_bad,std_part_of_bad,part_of_not_bad,std_part_of_not_bad,log_part_of_not_bad
0,3,test,4,0,0,4,0.000000,NaN,0.000000,NaN,1.000000,NaN,0.000000
1,5,ctrl,2,0,0,2,0.000000,NaN,0.000000,NaN,1.000000,NaN,0.000000
2,5,test,1,1,0,2,0.500000,0.750000,0.000000,NaN,1.000000,NaN,0.000000
3,6,ctrl,5,0,1,6,0.000000,NaN,0.166667,0.194444,0.833333,0.194444,-0.182322
4,6,test,7,2,1,10,0.200000,0.120000,0.100000,0.110000,0.900000,0.110000,-0.105361
5,7,ctrl,5,0,1,6,0.000000,NaN,0.166667,0.194444,0.833333,0.194444,-0.182322
6,7,test,3,2,1,6,0.333333,0.222222,0.166667,0.194444,0.833333,0.194444,-0.182322
7,8,ctrl,7,1,2,10,0.100000,0.110000,0.200000,0.120000,0.800000,0.120000,-0.223144
8,8,test,6,3,3,12,0.250000,0.104167,0.250000,0.104167,0.750000,0.104167,-0.287682
9,9,ctrl,10,3,1,14,0.214286,0.086735,0.071429,0.076531,0.928571,0.076531,-0.074108


In [89]:
fig = px.scatter(
    session_len_grouped[(session_len_grouped["cnt"] >= 5)], 
    y='part_of_good', 
    x='session_len', 
    color="test_name")
fig.show()

In [90]:
fig = px.scatter(
    session_len_grouped[(session_len_grouped["cnt"] >= 5)], 
    y='part_of_bad', 
    x='session_len', 
    color="test_name")
fig.show()

In [91]:
fig = px.scatter(
    session_len_grouped[(session_len_grouped["cnt"] >= 5)], 
    y='part_of_not_bad', 
    x='session_len', 
    color="test_name")
fig.show()

In [92]:
fig = px.scatter(
    session_len_grouped[(session_len_grouped["cnt"] >= 5)], 
    y='log_part_of_not_bad', 
    x='session_len', 
    color="test_name")
fig.show()